## Transform Orders Data - String to JSON
1. Pre-process the JSON String to fix the Data Quality Issues
1. Transform JSON String to JSON Object
1. Write transformed data to the silver schema

In [0]:
SELECT * 
  FROM gizmobox.bronze.v_orders;

### 1. Pre-process the JSON String to fix the Data Quality Issues
[regexp_replace function](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/functions/regexp_replace)

In [0]:
CREATE OR REPLACE TEMP VIEW tv_orders_fixed
AS
SELECT value,
       regexp_replace(value, '"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "\$1"') AS fixed_value 
FROM gizmobox.bronze.v_orders;

### 2. Transform JSON String to JSON Object
- Function [schema_of_json](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/functions/schema_of_json)
- Function [from_json](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/functions/from_json)

In [0]:
SELECT schema_of_json(fixed_value) AS schema,
       fixed_value
FROM tv_orders_fixed
LIMIT 1;

In [0]:
SELECT from_json(fixed_value,
       'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') AS json_value,
       fixed_value
FROM tv_orders_fixed;

### 3. Write transformed data to the silver schema

In [0]:
CREATE TABLE IF NOT EXISTS gizmobox.silver.orders_json
AS
SELECT from_json(fixed_value,
       'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') AS json_value
FROM tv_orders_fixed;

In [0]:
SELECT * FROM gizmobox.silver.orders_json;